In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Read the data
X = pd.read_csv('../input/train.csv', index_col='Id')
X_test_full = pd.read_csv('../input/test.csv', index_col='Id')

# Remove rows with missing target, separate target from predictors
X.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X.SalePrice              
X.drop(['SalePrice'], axis=1, inplace=True)

# Break off validation set from training data
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2,
                                                                random_state=0)
# # "Cardinality" means the number of unique values in a column
# # Select categorical columns with relatively low cardinality (convenient but arbitrary)
# low_cardinality_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and 
#                         X_train_full[cname].dtype == "object"]

# Select numeric columns
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]
X_num = X_train_full[numeric_cols].copy()
X_valid_num = X_valid_full[numeric_cols].copy()
from sklearn.impute import SimpleImputer
my_imputer =  SimpleImputer(strategy='median')
X_train_imp = pd.DataFrame(my_imputer.fit_transform(X_num))
X_valid_imp = pd.DataFrame(my_imputer.transform(X_valid_num))
X_train_imp.columns = X_num.columns
X_valid_imp.columns = X_valid_num.columns

# Test Data Preprocessing
imputer =  SimpleImputer(strategy='median')
X_test_num = X_test_full[numeric_cols].copy()
X_test_imp = pd.DataFrame(imputer.fit_transform(X_test_num))
X_test_imp.columns = X_test_num.columns

cat_cols = [col for col in X_train_full.columns if X_train_full[col].dtype in ['object']]
X_train_imp[cat_cols] = X_train_full[cat_cols]
X_train_prep = X_train_imp.copy()
X_valid_imp[cat_cols] = X_valid_full[cat_cols]
X_valid_prep = X_valid_imp.copy()
X_test_imp[cat_cols] = X_test_full[cat_cols]
X_test_prep = X_test_imp.copy()
X_test_prep.index = X_test_num.index

# Keep selected columns only
# my_cols = cat_cols + numeric_cols
# X_train = X_train_prep[my_cols].copy()
# X_valid = X_valid_prep[my_cols].copy()
# X_test = X_test_prep[my_cols].copy()

# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train_prep)
X_valid = pd.get_dummies(X_valid_prep)
X_test = pd.get_dummies(X_test_prep)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)
X_train, X_test = X_train.align(X_test, join='left', axis=1)

# Final Model


In [2]:
final_X = X_train.append(X_valid)
final_y = y_train.append(y_valid)

In [3]:
from xgboost import XGBRegressor
my_model = XGBRegressor(n_estimators=950, learning_rate=0.07, max_depth=2)
my_model.fit(final_X, final_y)

XGBRegressor(base_score=0.5, booster=None, colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints=None,
             learning_rate=0.07, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             n_estimators=950, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
             validate_parameters=False, verbosity=None)

In [4]:
preds_test = my_model.predict(X_test)

In [5]:
# Save test predictions to file
output = pd.DataFrame({'Id': X_test.index,
                       'SalePrice': preds_test})
output.to_csv('submission.csv', index=False)